In [1]:
import pandas as pd 
import numpy as np
import math
from demand_allocation_dict import demand_allocation

from extensions import disaggregate_energy_and_vehs, determine_nb_of_charging_vehs_per_day, capacity_sizing, determine_total_hours_of_charging, hourly_dissagregation, creating_hour_demand_dict, adding_hours_demand, create_total_yearly_demand_profile

8760  time steps


In [2]:
demand_allocation
print(demand_allocation)

{'night_charging': {'share': 1, 'start_time': 17, 'end_time': 7, 'strategy_split': {'even': 0.5, 'earliest': 0.3, 'latest': 0.2}, 'charging_power': 11, 'cars_per_cs_per_day': 1}, 'day_quick_charging': {'share': 0, 'start_time': 10, 'end_time': 16, 'strategy_split': {'even': 1, 'earliest': 0, 'latest': 0}, 'charging_power': 50, 'cars_per_cs_per_day': 8}}


In [3]:
batt_cap = 50   # kWh
total_vehs = 1000
total_demand = total_vehs * 365 * batt_cap/3 

year_to_be_modeled = 2030
total_charging_shares = sum(demand_allocation[da]["share"] for da in demand_allocation.keys())
nb_days = 366 if (year_to_be_modeled % 4 == 0 and (year_to_be_modeled % 100 != 0 or year_to_be_modeled % 400 == 0)) else 365
daily_demand = total_demand / nb_days
assert round(total_charging_shares) == 1, "The sum of the shares must be equal to 1"

In [4]:
charging_strategy = list(demand_allocation.keys())[0]
share = demand_allocation[charging_strategy]["share"]
start_time = demand_allocation[charging_strategy]["start_time"]
end_time = demand_allocation[charging_strategy]["end_time"]
charging_power = demand_allocation[charging_strategy]["charging_power"]
cars_per_cs_per_day = demand_allocation[charging_strategy]["cars_per_cs_per_day"]

# step 1: determine the number of vehs and energy for this charging strategy

In [5]:
curr_energy_demand, vehs = disaggregate_energy_and_vehs(total_demand, total_vehs, share)
print(total_demand, curr_energy_demand, vehs)

curr_daily_demand = daily_demand * share
print(curr_daily_demand, curr_energy_demand/365)

6083333.333333333 6083333.333333333 1000
16666.666666666664 16666.666666666664


# step 2: determine the number of vehs to be charged per day

In [6]:
vehs_to_be_charged_per_day = determine_nb_of_charging_vehs_per_day(vehs, batt_cap, curr_daily_demand/vehs)
print(vehs_to_be_charged_per_day)

416.66666666666663


# step 3: determine the capacity of the charging infrastructure

In [7]:
charging_infra_capacity = capacity_sizing(vehs_to_be_charged_per_day, cars_per_cs_per_day, charging_power)
print(charging_infra_capacity)
print(vehs_to_be_charged_per_day * 11)

4583.333333333333
4583.333333333333


# step 4: determine the hourly demand profile

In [8]:
charging_hours = determine_total_hours_of_charging(start_time, end_time)
print(charging_hours)

14


In [9]:
mode = "even" 
curr_energy_demand_by_mode = curr_energy_demand * demand_allocation[charging_strategy]["strategy_split"][mode]
print(curr_energy_demand_by_mode, curr_energy_demand)
hour_profile = hourly_dissagregation(curr_energy_demand_by_mode/nb_days, charging_infra_capacity, charging_hours, mode=mode)
print(hour_profile, charging_infra_capacity)

3041666.6666666665 6083333.333333333
[595.23809524 595.23809524 595.23809524 595.23809524 595.23809524
 595.23809524 595.23809524 595.23809524 595.23809524 595.23809524
 595.23809524 595.23809524 595.23809524 595.23809524] 4583.333333333333


In [10]:
mode = "latest" 
curr_energy_demand_by_mode = curr_energy_demand * demand_allocation[charging_strategy]["strategy_split"][mode]
print(curr_energy_demand_by_mode, curr_energy_demand)
hour_profile = hourly_dissagregation(curr_energy_demand_by_mode/nb_days, charging_infra_capacity, charging_hours, mode=mode)
print(curr_energy_demand_by_mode/nb_days, charging_infra_capacity)
print(hour_profile)

1216666.6666666667 6083333.333333333
3333.3333333333335 4583.333333333333
[   0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.         3333.33333333]


# step 4: determine the hourly demand profile

In [11]:
hour_profiles = []
print(curr_energy_demand)
for mode in demand_allocation[charging_strategy]["strategy_split"]:
    curr_energy_demand_by_mode = curr_energy_demand * demand_allocation[charging_strategy]["strategy_split"][mode]
    hour_profile = hourly_dissagregation(curr_energy_demand_by_mode/nb_days, charging_infra_capacity, charging_hours, mode=mode)
    hour_profiles.append(hour_profile)
print(hour_profiles)

6083333.333333333
[array([595.23809524, 595.23809524, 595.23809524, 595.23809524,
       595.23809524, 595.23809524, 595.23809524, 595.23809524,
       595.23809524, 595.23809524, 595.23809524, 595.23809524,
       595.23809524, 595.23809524]), array([4583.33333333,  416.66666667,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ]), array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        , 3333.33333333])]


# step 5: make dictionary of hourly demand

In [12]:
hourly_profiles_total = []
hour_profiles_dicts = []
for h_profile in hour_profiles:
    hour_profile_dict = creating_hour_demand_dict(start_time, end_time, h_profile)
    hour_profiles_dicts.append(hour_profile_dict)
    hourly_profiles_total.append(hour_profile_dict)
    print(hour_profile_dict)

# print(hourly_profiles_total)

{0: np.float64(595.2380952380952), 1: np.float64(595.2380952380952), 2: np.float64(595.2380952380952), 3: np.float64(595.2380952380952), 4: np.float64(595.2380952380952), 5: np.float64(595.2380952380952), 6: np.float64(595.2380952380952), 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: np.float64(595.2380952380952), 18: np.float64(595.2380952380952), 19: np.float64(595.2380952380952), 20: np.float64(595.2380952380952), 21: np.float64(595.2380952380952), 22: np.float64(595.2380952380952), 23: np.float64(595.2380952380952)}
{0: np.float64(0.0), 1: np.float64(0.0), 2: np.float64(0.0), 3: np.float64(0.0), 4: np.float64(0.0), 5: np.float64(0.0), 6: np.float64(0.0), 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: np.float64(4583.333333333333), 18: np.float64(416.66666666666606), 19: np.float64(0.0), 20: np.float64(0.0), 21: np.float64(0.0), 22: np.float64(0.0), 23: np.float64(0.0)}
{0: np.float64(0.0), 1: np.float64(0.0), 2: np.float64(0.0), 3: n

In [13]:
hour_profile = hour_profiles[0]
hour_demand_profile = {hour: 0 for hour in range(24)}
if end_time < start_time:
    fake_end_time = end_time + 24
else:
    fake_end_time = end_time
print(fake_end_time)
for hour in range(start_time, fake_end_time):
    charging_demand = hour_profile[hour-start_time]
    if hour > 23:
        hour_demand_profile[hour - 24] = charging_demand
    else:
        hour_demand_profile[hour] = charging_demand

print(hour_demand_profile)


31
{0: np.float64(595.2380952380952), 1: np.float64(595.2380952380952), 2: np.float64(595.2380952380952), 3: np.float64(595.2380952380952), 4: np.float64(595.2380952380952), 5: np.float64(595.2380952380952), 6: np.float64(595.2380952380952), 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: np.float64(595.2380952380952), 18: np.float64(595.2380952380952), 19: np.float64(595.2380952380952), 20: np.float64(595.2380952380952), 21: np.float64(595.2380952380952), 22: np.float64(595.2380952380952), 23: np.float64(595.2380952380952)}


# final step: summing up all profiles

In [14]:
hourly_demand_dict = adding_hours_demand(hourly_profiles_total)
print(hourly_demand_dict)


{0: np.float64(595.2380952380952), 1: np.float64(595.2380952380952), 2: np.float64(595.2380952380952), 3: np.float64(595.2380952380952), 4: np.float64(595.2380952380952), 5: np.float64(595.2380952380952), 6: np.float64(3928.5714285714284), 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: np.float64(5178.571428571428), 18: np.float64(1011.9047619047612), 19: np.float64(595.2380952380952), 20: np.float64(595.2380952380952), 21: np.float64(595.2380952380952), 22: np.float64(595.2380952380952), 23: np.float64(595.2380952380952)}


In [15]:
total_yearly_profile = create_total_yearly_demand_profile(hourly_demand_dict, nb_days)
print(len(total_yearly_profile), " time steps")

8760  time steps
